## Pivin R. Homework. 
## Churn analysis
### HSE, Data-driven 21, May'22


# Pre-work <a class="anchor" id="pre"></a>

At this stage we load all packages required

In [ ]:
#conda install -c conda-forge xgboost

In [ ]:
import pandas as pd
import numpy as np # Packages for data wrangling and modelling

import matplotlib.pyplot as plt
import seaborn as sns # Packages for visualization

from xgboost import XGBClassifier 

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, average_precision_score
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold, RandomizedSearchCV 
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.preprocessing import OneHotEncoder


In [ ]:
df = pd.read_csv('/Users/rostislav/opt/anaconda3/notebooks/HSE Data-driven/Datasets/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:
df.head()

# Preparation and exploration <a class="anchor" id="expl"></a>

### Sanity checks

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.customerID.nunique() == len(df)

No duplicates allready

In [ ]:
df.Churn.value_counts()

The sample is unbalanced!

In [ ]:
#for column in df.columns:
#    print(df[column].value_counts())
#    print('\n')

#### Numeric features

In [ ]:
df.TotalCharges

In [ ]:
to_drop = []
for n,value in enumerate(df.TotalCharges):
    try: float(value)
    except: to_drop.append(n)
df = df.drop(df.index[to_drop])
df.TotalCharges = df.TotalCharges.astype(float)

#### Nominal features

Features "Internet service" and "Phone Service" are detailed in few others. E.g. Phone service detailed in the "Multiple lines". They, by definitions, would be highly correlated which will make the regression is statistically insignificant. So, we should either keep only the main features or the detailed ones. Let's try both approaches.

In [ ]:
df = df.drop(['customerID'],axis=1)
df = df.dropna()

In [ ]:
df_main = df.drop(['MultipleLines','OnlineSecurity','OnlineBackup',
                   'DeviceProtection','TechSupport','StreamingTV',
                   'StreamingMovies'],axis=1)
df_detailed = df.drop(['PhoneService','InternetService'],axis=1)

### Getting dummies

In [ ]:
df_main = df_main.replace('No',0)
df_main = df_main.replace('Yes',1)

df_detailed = df_detailed.replace('No',0)
df_detailed = df_detailed.replace('Yes',1)
df_detailed = df_detailed.replace('No internet service',0)
df_detailed = df_detailed.replace('No phone service',0)

In [ ]:
df_detailed.head()

In [ ]:
dummies_main = pd.get_dummies(df[['gender','InternetService','Contract','PaymentMethod']])
df_main = pd.concat([df_main, dummies_main], axis=1)
df_main = df_main.drop(['gender','InternetService','Contract','PaymentMethod'],axis=1)

dummies_detailed = pd.get_dummies(df[['gender','Contract','PaymentMethod']])
df_detailed = pd.concat([df_detailed, dummies_detailed], axis=1)
df_detailed = df_detailed.drop(['gender','Contract','PaymentMethod'],axis=1)

### Multicollinearity check

In [ ]:
plt.matshow(df_main.corr())
corr_matrix = df_main.corr(method='pearson')

In [ ]:
plt.matshow(df_detailed.corr())
corr_matrix = df_detailed.corr(method='pearson')

Everything is ok, we can proceed

# XGboost classifier implementation <a class="anchor" id="model"></a>

## Main

In [ ]:
#regressors and key variable
X = df_main.drop("Churn",axis=1)
y = df_main.Churn

#train-test
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

#shuffling
data = df_main.sample(frac=1)
X = df_main.drop("Churn",axis=1)
y = df_main.Churn

In [ ]:
#model 
model = XGBClassifier()

#parameters
param_list = {
    'silent': [False],
    'max_depth': range(4,40),
    'learning_rate': [0.005, 0.01, 0.1, 0.2],
    'subsample': np.arange(0,1.2,.3),
    'colsample_bytree': np.arange(0,1.2,.3),
    'colsample_bylevel': np.arange(0,1.2,.3),
    'min_child_weight': [0.5, 1.0, 2.0],
    'gamma': [0, 0.25, 0.5, 0.75, 1.0],
    'reg_lambda': [0.1, 10.0, 50.0, 100.0, 500.0, 1000.0],
    'n_estimators': [10, 50],
    'scale_pos_weight': [1, 6],
    'max_delta_step': [1, 2, 3]
}
kfold = 10
cv_strat = RepeatedStratifiedKFold(n_splits=kfold,n_repeats=5)

#Randomized Search
cv = RandomizedSearchCV(model,param_list,cv=cv_strat,n_iter=10,verbose=1,scoring="balanced_accuracy",n_jobs=-1).fit(X.values,y.values)

model_best = cv.best_estimator_
cv.best_params_ # To see best hyperparameters found

Cross-validation

In [ ]:
y_pred_train = model_best.predict(X_train.values)
y_pred = model_best.predict(X_test.values)

cv_strat = RepeatedStratifiedKFold(n_splits=kfold,n_repeats=20)
scores = cross_validate(model_best,X.values,y.values,cv=cv_strat,verbose=3,n_jobs=-1,return_train_score=True,
                        scoring={"roc_auc":"roc_auc",
                                 "recall":"recall",
                                 "precision":"precision",
                                 "accuracy":"accuracy",
                                 "balanced_accuracy":"balanced_accuracy",
                                 "average_precision":"average_precision"}) 

stat_xgb = pd.DataFrame(pd.DataFrame(scores).mean(),columns=["Score_main"]).drop(["fit_time","score_time"])


In [ ]:
stat_xgb

Confusion matrix

In [ ]:
#generate a confusion matrix to visualise precision, recall, misclassification and false alarms
cm = pd.DataFrame(confusion_matrix(y_test, y_pred), index = list(set(y)), columns = list(set(y)))

#visualise the confusion matrix
plt.figure()
sns.heatmap(cm, annot = True, fmt="d",
            cmap=sns.color_palette("Blues")).set(xlabel='predicted values', 
                                                ylabel='real values', 
                                                title = 'Confusion Matrix')

In [ ]:
cm

While the metrics are ok, from confusion matrix we can see that model often predicts churn mistakenly 

Let's check feature importance

In [ ]:
feat_imp = pd.DataFrame(list(zip(data.columns,model_best.feature_importances_)),columns=["Feature","Importance"]).sort_values(by="Importance",ascending=False)
fig_store = plt.figure(figsize=(10,20))
sns.barplot(y="Feature",x="Importance",data = feat_imp,orient="h")
plt.show()

We can see, that only limited number of features is relevant for this model. Let's compare with the other model

## Detailed

In [ ]:
#regressors and key variable
X = df_detailed.drop("Churn",axis=1)
y = df_detailed.Churn

#train-test
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

#shuffling
data = df_detailed.sample(frac=1)
X = df_detailed.drop("Churn",axis=1)
y = df_detailed.Churn

#model 
model = XGBClassifier()

#parameters
param_list = {
    'silent': [False],
    'max_depth': range(4,40),
    'learning_rate': [0.005, 0.01, 0.1, 0.2],
    'subsample': np.arange(0,1.2,.3),
    'colsample_bytree': np.arange(0,1.2,.3),
    'colsample_bylevel': np.arange(0,1.2,.3),
    'min_child_weight': [0.5, 1.0, 2.0],
    'gamma': [0, 0.25, 0.5, 0.75, 1.0],
    'reg_lambda': [0.1, 10.0, 50.0, 100.0, 500.0, 1000.0],
    'n_estimators': [10, 50],
    'scale_pos_weight': [1, 6],
    'max_delta_step': [1, 2, 3]
}
kfold = 10
cv_strat = RepeatedStratifiedKFold(n_splits=kfold,n_repeats=5)

#Randomized Search
cv = RandomizedSearchCV(model,param_list,cv=cv_strat,n_iter=10,verbose=1,scoring="balanced_accuracy",n_jobs=-1).fit(X.values,y.values)

model_best = cv.best_estimator_

y_pred_train = model_best.predict(X_train.values)
y_pred = model_best.predict(X_test.values)

cv_strat = RepeatedStratifiedKFold(n_splits=kfold,n_repeats=20)
scores = cross_validate(model_best,X.values,y.values,cv=cv_strat,verbose=3,n_jobs=-1,return_train_score=True,
                        scoring={"roc_auc":"roc_auc",
                                 "recall":"recall",
                                 "precision":"precision",
                                 "accuracy":"accuracy",
                                 "balanced_accuracy":"balanced_accuracy",
                                 "average_precision":"average_precision"}) 

stat_xgb_detailed = pd.DataFrame(pd.DataFrame(scores).mean(),columns=["Score"]).drop(["fit_time","score_time"])



In [ ]:
stat_xgb['Score_detaile'] = stat_xgb_detailed.Score
stat_xgb

In [ ]:
#generate a confusion matrix to visualise precision, recall, misclassification and false alarms
cm = pd.DataFrame(confusion_matrix(y_test, y_pred), index = list(set(y)), columns = list(set(y)))

#visualise the confusion matrix
plt.figure()
sns.heatmap(cm, annot = True, fmt="d",
            cmap=sns.color_palette("Blues")).set(xlabel='predicted values', 
                                                ylabel='real values', 
                                                title = 'Confusion Matrix')

In [ ]:
cm

In [ ]:
feat_imp = pd.DataFrame(list(zip(data.columns,model_best.feature_importances_)),columns=["Feature","Importance"]).sort_values(by="Importance",ascending=False)
fig_store = plt.figure(figsize=(10,20))
sns.barplot(y="Feature",x="Importance",data = feat_imp,orient="h")
plt.show()

 Detailed features enable much more important regressors, but each of them has limited importance. The model in general works significantly worse than general model

# Conclusion

After modelling we discover that it's almost vital for telecom to convince its customers to become their internet service provider. This is the most viable predictor of the churn. 

Among other features:
- the longer the contract the better
- Billing via internet is the best (and good for Earth)
- Gender has medium importance